# This code is to make .bin files to .tfrecords

## Mount on your drive to use data stored 

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

## Import required libraries

In [ ]:
import ntpath
import os
import glob
import struct
import tensorflow as tf # tensorflow 1.14, 2.0.0-alpha, 2.0.0-beta1
from tensorflow.core.example import example_pb2
import argparse

### Example generator

In [ ]:
def example_generator(file):
  
  while True:
    len_bytes = file.read(8)
    if not len_bytes: break # finished reading this file
    str_len = struct.unpack('q', len_bytes)[0]
    example_str = struct.unpack('%ds' % str_len, file.read(str_len))[0]
    yield example_pb2.Example.FromString(example_str)

In [ ]:

def art_abs_example(article, abstract):
  
  def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
      value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

  feature = {
      'article': _bytes_feature(article),
      'abstract': _bytes_feature(abstract)
  }

  return tf.train.Example(features=tf.train.Features(feature=feature))

### Main method

In [ ]:
def make_TFRecords(data_path, new_data_path):
  print("Starting ...")
  if not os.path.exists(new_data_path):
    os.makedirs(new_data_path)
  filelist = glob.glob(data_path+"/**/*.bin", recursive=True) # get the list of datafiles
  assert filelist, "No binary files"
  
  common_path = os.path.commonpath(filelist)
  files = [os.path.splitext(x.replace(common_path, ""))[0]  for x in filelist]
  
  for f, filename in zip(filelist, files):
    try:
      file =  open(f, 'rb')
    except:
      print("Cannot open file : {}".format(f))
      continue
     
    record_file = '{}/{}.tfrecords'.format(new_data_path, filename)
    record_dir = os.path.dirname(record_file)
    if not os.path.exists(record_dir):
      os.makedirs(record_dir)
    with tf.io.TFRecordWriter(record_file) as writer:
      for e in example_generator(file):
        try:
          article_text = e.features.feature['article'].bytes_list.value[0].decode()
          abstract_text = e.features.feature['abstract'].bytes_list.value[0].decode()
          
          tf_example = art_abs_example(article_text, abstract_text)
          writer.write(tf_example.SerializeToString())

        except ValueError:
          tf.logging.error('Failed to get article or abstract from example')
          continue
        if len(article_text) == 0   :
          tf.logging.warning('Found an example with empty article text. Skipping it.')
          
    print("Chunked file {} processed and saved to {}".format(f, record_file))

## Use case

In [ ]:
data_path2 = "/content/"
new_data_path = "tfrecords_folder1"

In [ ]:
make_TFRecords(data_path2, new_data_path)

Starting ...
Chunked file /content/val_000.bin processed and saved to tfrecords_folder1//val_000.tfrecords
Chunked file /content/val_001.bin processed and saved to tfrecords_folder1//val_001.tfrecords


In [ ]:
ls $new_data_path

val_000.tfrecords  val_001.tfrecords


# Save files to local

In [ ]:
from google.colab import files


In [ ]:
!zip -r /content/tf.zip /content/tfrecords_folder

  adding: content/tfrecords_folder/ (stored 0%)
  adding: content/tfrecords_folder/test_001.tfrecords (deflated 72%)
  adding: content/tfrecords_folder/test_000.tfrecords (deflated 73%)


In [ ]:
files.download('/content/tf.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>